In [ ]:
import numpy as np
from scipy.stats import norm, multivariate_normal
import pandas as pd

#  Экзотические опционы

In [ ]:
# вспомогательные функции для аппроксимации двумерного нормального распределения
def f(x, y, a, b, rho):
    return np.exp(_a(a, rho)*(2*x - _a(a, rho)) + _b(b, rho)*(2*y - _b(b, rho)) + 2*rho*(x-_a(a, rho))*(y-_b(b, rho)))

def _a(a, p):
    return a/np.sqrt(2*(1-p**2))

def _b (b, p):
    return b/np.sqrt(2*(1-p**2))


def M(a, b, rho):
    A = [0.3253030,  0.4211071, 0.1334425, 0.006374323]
    B = [0.1337764,  0.6243247, 1.3425378, 2.2626645]
    sum = 0
    for i in range(len(A)):
        for j in range(len(B)):
            sum += A[i]*A[j]*f(B[i], B[j], a, b, rho)

    return np.sqrt(1-rho**2)/np.pi * sum

In [ ]:
def OptionBlackSholes(call_or_put, S, K, T, r, b, sigma):
    d1 = (np.log(S / K) + (b + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if call_or_put == 'call':
      c = S * np.exp((b-r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
      return c
    elif call_or_put == 'put':
      p = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp((b-r) * T) * norm.cdf(-d1)
      return p

In [ ]:
# Example. Answer is 4.0870
OptionBlackSholes('put', 75, 70, 0.5, 0.1, 0.05, 0.35)

4.086953828635355

In [ ]:
def OptionDelta(CallPutFlag, S, K, T, r, b, sigma):
    d1 = (np.log(S/K) + (b + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    if CallPutFlag == 'call':
      c_delta = np.exp((b-r) * T) * norm.cdf(d1)
      return c_delta
    elif CallPutFlag == 'put':
      p_delta = -np.exp((b-r) * T) * norm.cdf(-d1)
      return p_delta

In [ ]:
# Example. Answer is 1.1273
OptionDelta('call', 90, 40, 2, 0.03, 0.09, 0.2)

1.1273460446414625

##  Опцоны с выбором (Chooser options)
Пример 1. Оцените справедливую стоимость опциона, который по истечению трех месяцев дает держателю право выбора: между шестимесячным опционом колл со страйком 55 и семимесячным опционом пут со страйком 48. Базовая цена акций — 50, безрисковая процентная ставка 10% годовых, дивидендная доходность — 5% годовых, а годовая волатильность 35% [Haug 2007, p.130].

In [ ]:
S = 50
K_c = 55
K_p = 48
T_c = 1/2  # 0.5
T_p = 7/12 # 0.5833
t = 0.25
r = 0.1
b = 0.1 - 0.05
sigma = 0.35

In [ ]:
# решение уравнения с помощью метода Ньютона-Рафсона

def CriticalValueChooser(S, K_c, K_p, T, T_c, T_p, r, b, sigma):
    epsilon = 1e-3

    Sv = S
    c_i = OptionBlackSholes('call', Sv, K_c, K_c - T, r, b, sigma)
    p_i = OptionBlackSholes('put', Sv, K_p, T_p - T, r, b, sigma)
    d_c = OptionDelta('call', Sv, K_c, T_c - T, r, b, sigma)
    d_p = OptionDelta('put', Sv, K_p, T_p - T, r, b, sigma)
    y_i = c_i - p_i
    d_i = d_c - d_p

    while abs(y_i) > epsilon:
        Sv = Sv - y_i / d_i
        c_i = OptionBlackSholes('call', Sv, K_c, T_c - T, r, b, sigma)
        p_i = OptionBlackSholes('put', Sv, K_p, T_p - T, r, b, sigma)
        d_c = OptionDelta('call', Sv, K_c, T_c - T, r, b, sigma)
        d_p = OptionDelta('put', Sv, K_p, T_p - T, r, b, sigma)
        y_i = c_i - p_i
        d_i = d_c - d_p

    return Sv

In [ ]:
# оценка справедливой стоимости,  w опциона с выбором
def ComplexChooser(S, K_c, K_p, t, T_c, T_p, r, b, sigma):
    I = CriticalValueChooser(S, K_c, K_p, t, T_c, T_p, r, b, sigma)
    # print('решение уравнения, I= ', I)

    d1 = (np.log(S / I) + (b + sigma ** 2 / 2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)

    y1 = (np.log(S / K_c) + (b + sigma ** 2 / 2) * T_c) / (sigma * np.sqrt(T_c))
    y2 = (np.log(S / K_p) + (b + sigma ** 2 / 2) * T_p) / (sigma * np.sqrt(T_p))

    p1 = np.sqrt(t / T_c)
    p2 = np.sqrt(t / T_p)

    w = S * np.exp((b-r)*T_c) * M(d1, y1, p1) - \
      K_c * np.exp(-r*T_c) * M(d2, y1 - sigma * np.sqrt(T_c), p1) - \
        S * np.exp((b-r) * T_p) * M(-d1, -y2, p2) + \
            K_p * np.exp(-r * T_p) * M(-d2, -y2 + sigma * np.sqrt(T_p), p2)
    #print('оценка справедливой стоимости = ', w)

    return w

In [ ]:
# Example. Answer is 6.0508
ComplexChooser(S, K_c, K_p, t, T_c, T_p, r, b, sigma)

6.00391974201559

## Составные опционы

Пример 2. Оцените стоимость опциона «пут-на-колл», который дает держателю опциона право продать опцион «колл» за 50 через три месяца. Страйк базового опциона колл составляет 520, срок погашения опциона колл - через шесть месяцев, цена базового актива составляет 500, безрисковая процентная ставка составляет 8%, дивидендная доходность 3% годовых и волатильность 35%.

In [ ]:
S = 500
K1 = 520
K2 = 50
t1 = 1/4
T2 = 1/2
г = 0.08
b = 0.08-0.03
sigma  = 0.35

In [ ]:
# решение уравнения с помощью метода Ньютона-Рафсона
def CriticalValueOptionsOnOptions(CallPutFlag, K1, K2, T, r, b, sigma):
    Si = K1
    ci = OptionBlackSholes(CallPutFlag, Si, K1, T, r, b, sigma)
    di = OptionDelta(CallPutFlag, Si, K1, T, r, b, sigma)
    epsilon = 1e-6

    while abs(ci - K2) > epsilon:
        Si = Si - (ci - K2) / di
        ci = OptionBlackSholes(CallPutFlag, Si, K1, T, r, b, sigma)
        di = OptionDelta(CallPutFlag, Si, K1, T, r, b, sigma)

    return Si

In [ ]:
# оценка справедливой стоимости составного опциона

def OptionsOnOptions(TypeFlag, S, K1, K2, t1, T2, r, b, sigma):

    if TypeFlag == 'call-on-call' or TypeFlag == 'put-on-call':
        CallPutFlag = 'call'
    else:
        CallPutFlag = 'put'

    i = CriticalValueOptionsOnOptions(CallPutFlag, K1, K2, T2 - t1, r, b, sigma)

    rho = np.sqrt(t1/T2)

    y1 = (np.log(S / i) + (b + sigma ** 2 / 2) * t1) / (sigma * np.sqrt(t1))
    y2 = y1 - sigma * np.sqrt(t1)
    z1 = (np.log(S/K1) + (b + sigma **2 / 2) * T2) / (sigma * np.sqrt(T2))
    z2 = z1 - sigma * np.sqrt(T2)

    if TypeFlag == 'call-on-call':
        return S * np.exp((b-r) * T2) * M(z1, y1, rho) - \
            K1 * np.exp(-r * T2) * M(z2, y2, rho) - \
                K2 * np.exp(-r * t1) * norm.cdf(y2)
    elif TypeFlag == 'put-on-call':
        return K1 * np.exp(-r * T2) * M(z2, -y2, -rho) - \
            S * np.exp((b-r) * T2) * M(z1, -y1, -rho) + \
                K2 * np.exp(-r * t1) * norm.cdf(-y2)
    elif TypeFlag == 'call-on-put':
        return  K1 * np.exp(-r * T2) * M(-z2, -y2, rho) - \
            S * np.exp((b-r) * T2) * M(-z1, -y1, rho) - \
                K2 * np.exp(-r * t1) * norm.cdf(-y2)
    elif TypeFlag == 'put-on-put':
        return S * np.exp((b-r) * T2) * M(-z1, y1, -rho) - \
            K1 * np.exp(-r * T2) * M(-z2, y2, -rho) + \
                np.exp(-r * t1) * K2 * norm.cdf(y2)

In [ ]:
# Example. Answer is 21.1965
OptionsOnOptions('put-on-call', 500, 520, 50, 1/4, 1/2, 0.08, 0.05, 0.35)

21.196205693039495

In [ ]:
# В условиях примера 2 написать функцию для оценки стоимости сложного опциона с использованием встроенной функции
# multivariate_normal.pdf() и сравнить результаты

def OptionsOnOptions_CBND(TypeFlag, S, K1, K2, t1, T2, r, b, sigma):
    if TypeFlag == 'call-on-call' or TypeFlag == 'put-on-call':
        CallPutFlag = 'call'
    else:
        CallPutFlag = 'put'

    #S_star = CriticalValueOptionsOnOptions(CallPutFlag, K1, K2, abs(T2 - t1), r, b, sigma)
    S_star = 90
    rho = np.sqrt(t1/T2)
    sigma1 = 1
    sigma2 = 1
    mu1 = 0.0
    mu2 = 0.0
    y1 = (np.log(S / S_star) + (b + sigma ** 2 / 2) * t1) / (sigma * np.sqrt(t1))
    y2 = y1 - sigma * np.sqrt(t1)
    z1 = (np.log(S/K1) + (b + sigma **2 / 2) * T2) / (sigma * np.sqrt(T2))
    z2 = z1 - sigma * np.sqrt(T2)
    if TypeFlag == 'call-on-call':
        cov_matrix = np.array([[1 * sigma1**2, rho * sigma1 * sigma2],
                            [rho * sigma1 * sigma2, 1 * sigma2**2]])
        mv1 = multivariate_normal.cdf([z1, y1],[mu1, mu2] , cov_matrix)
        mv2 = multivariate_normal.cdf([z2, y2],[mu1, mu2] , cov_matrix)
        return S * np.exp((b-r) * T2) * mv1- \
            K1 * np.exp(-r * T2) * mv2 - \
                K2 * np.exp(-r * t1) * norm.cdf(y2)
    elif TypeFlag == 'put-on-call':
        cov_matrix = np.array([[1 * sigma1**2, -rho * sigma1 * sigma2],
                            [-rho * sigma1 * sigma2, 1 * sigma2**2]])
        mv1 = multivariate_normal.cdf([z2, -y2],[mu1, mu2] , cov_matrix)
        mv2 = multivariate_normal.cdf([z1, -y1],[mu1, mu2] , cov_matrix)
        return K1 * np.exp(-r * T2) * mv1 - \
            S * np.exp((b-r) * T2) * mv2 + \
                K2 * np.exp(-r * t1) * norm.cdf(-y2)
    elif TypeFlag == 'call-on-put':
        cov_matrix = np.array([[1 * sigma1**2, rho * sigma1 * sigma2],
                            [rho * sigma1 * sigma2, 1 * sigma2**2]])
        mv1 = multivariate_normal.cdf([-z2, -y2],[mu1, mu2] , cov_matrix)
        mv2 = multivariate_normal.cdf([-z1, -y1],[mu1, mu2] , cov_matrix)
        return  K1 * np.exp(-r * T2) * mv1 - \
            S * np.exp((b-r) * T2) * mv2 - \
                K2 * np.exp(-r * t1) * norm.cdf(-y2)
    elif TypeFlag == 'put-on-put':
        cov_matrix = np.array([[1 * sigma1**2, -rho * sigma1 * sigma2],
                            [-rho * sigma1 * sigma2, 1 * sigma2**2]])

        mv1 = multivariate_normal.cdf([-z1, y1],[mu1, mu2] , cov_matrix)
        mv2 = multivariate_normal.cdf([-z2, y2],[mu1, mu2] , cov_matrix)

        return S * np.exp((b-r) * T2) * mv1 - \
            K1 * np.exp(-r * T2) * mv2 + \
                np.exp(-r * t1) * K2 * norm.cdf(y2)


In [ ]:
# Example. Answer is 21.1965
OptionsOnOptions('put-on-call', 500, 520, 50, 1/4, 1/2, 0.08, 0.05, 0.35) , OptionsOnOptions_CBND('put-on-call', 500, 520, 50, 1/4, 1/2, 0.08, 0.05, 0.35)

(21.196205693039495, 21.196350394352358)

Задача 1. Предположим, что применима модель Блэка-Шоулза. Текущая цена акции 80, по ним выплачиваются дивиденды по ставке 2\%, волатильность 30\%, безрисковая процентная ставка составляет 6\%. Рассмотрим сложный опцион колл на пут. Срок действия базового опциона пут истекает через 4 года, а цена исполнения составляет 90. Срок действия сложного опциона истекает через один год, цена исполнения составляет 13, а стоимость составляет 5,20. Найдите цену составного опциона пут с тем же базовым опционом, что и составной колл.

In [ ]:
S = 80
b = 0.04
sigma = 0.3
r = 0.06
T2 = 4
K2 = 13
T1 = 1
K1 = 90
option_type1 = 'call-on-put'
option_type2 = 'put-on-put'
#option_price = 5.20
option1 = OptionsOnOptions_CBND(option_type1, S, K1, K2, T1, T2 - T1, r, b, sigma)
option2 = OptionsOnOptions_CBND(option_type2, S, K1, K2, T1, T2 - T1, r, b, sigma)
print(option_type1, option1)
print(option_type2, option2)

call-on-put 5.206781031593616
put-on-put 2.1060836765455235


## Опционы с последействием, плавающий страйк

Пример 3. В рамках можели Блэка-Шоулза оцените опционы с последейстием на недивидендные акции. Если цена акции равна 50, волатильность цены акции составляет 40% в год, безрисковая ставка составляет 10% годовых, срок погашения - 3 месяца.

In [ ]:
q = 0
r = 0.1
b = r - q
S0 = 50
sigma = 0.4
T = 3 / 12

In [ ]:
Smin = Smax = S0
# оценка справедливой стоимости лукбек опциона колл с плавающим страйком
def FloatingLookback_Call(S0, Smin, T, r, q, sigma):
  a1 = (np.log(S0 /Smin) + (r - q + 0.5 * sigma ** 2) * T) / \
           (sigma * np.sqrt(T))
  a2 = a1 - sigma * np.sqrt(T)
  a3 = (np.log(S0 /Smin) + (-r + q + 0.5 * sigma ** 2) * T) / \
           (sigma * np.sqrt(T))
  Y1 = -(2 / (sigma ** 2)) * (r - q - 0.5 * sigma ** 2) * np.log(S0 / Smin)
  z1 = 0.5 * sigma ** 2 / (r - q)
  c_fl = S0 * np.exp(-q * T) * (norm.cdf(a1) - z1 * norm.cdf(-a1)) - \
       Smin * np.exp(-r * T) * (norm.cdf(a2) - z1 * np.exp(Y1) * norm.cdf(-a3))
  return c_fl

# оценка справедливой стоимости лукбек опциона пут с плавающим страйком
def FloatingLookback_Put(S0, Smax, T, r, q, sigma):
  b1 =  (np.log(Smax/S0) + (-r + q + 0.5 * sigma ** 2) * T) / \
           (sigma * np.sqrt(T))

  b2 = b1- sigma * np.sqrt(T)
  b3 = (np.log(Smax / S0) + (r - q - 0.5 * sigma ** 2) * T) / \
           (sigma * np.sqrt(T))
  Y2 = (2 / (sigma ** 2)) * (r - q - 0.5 * sigma ** 2) * np.log(Smax / S0)
  z1 = 0.5 * sigma ** 2 / (r - q)
  p_fl = Smax * np.exp(-r * T) * (norm.cdf(b1) - z1 * np.exp(Y2) * norm.cdf(-b3)) + \
     S0 * np.exp(-q * T) * (z1 * norm.cdf(-b2) - norm.cdf(b2))
  return p_fl

print('Floating-strike call price:', FloatingLookback_Call(S0, Smin, T, r, q, sigma))
print('Floating-strike put price:',  FloatingLookback_Put(S0, Smax, T, r, q, sigma))

Floating-strike call price: 8.037120139607019
Floating-strike put price: 7.790219259890344


Задача. Допустим, что имеется лукбек опцион колл с датой экпирации через шесть месяцев. Данный опцион дает право купить базовый актив (фондовый индекс) по самой низкой цене, наблюдаемой в течение срока действия опциона. Минимальная значение фондового индекса, наблюдаемое на данный момент, составляет 100, цена базового актива - 120, безрисковая процентная ставка - 10%, дивидендная доходность — 6%, волатильность — 30%. Оцените справедливую стоимость опциона с плавающим страйком (Hang, 2007, p. 143).

In [ ]:
r = 0.1
q = 0.06
S = 120
sigma = 0.3
T = 1/2
Smax = 120
Smin = 100
print('Floating-strike call price:', FloatingLookback_Call(S, Smin, T, r, q, sigma))


Floating-strike call price: 25.353355271810187


Задача. В условиях примера 3 найти оценку справедливой стоимости лукбек опциона пут с плавающим страйком через паритет опционов и сравнить результаты.

In [ ]:
print('Floating-strike call price:', FloatingLookback_Call(S, Smin, T, r, q, sigma))
print('Floating-strike put price:', FloatingLookback_Put(S, Smax, T, r, q, sigma))
print('put option parity price', FloatingLookback_Call(S, Smin, T, r, q, sigma) - np.exp(-q*T) * S  + np.exp(-r*T) * Smin)


Floating-strike call price: 25.353355271810187
Floating-strike put price: 19.724502072028894
put option parity price 4.022833696060616


##  Опционы с последействием, фиксированный страйк

Пример 4 ([Nang, 2007, p. 145](https://drive.google.com/file/d/1yxjRkchVqvl2xkQFyeB2BKNQ1SKJSTtK/view?usp=drive_link)). Заполните таблицу для лукбек опционов колл и пут с фиксированным страйком для разных значений дат экспираций T = 0,5, 1,0, цен исполнения K = 95, 100, 105 и волатильностями σ = 0,1, 0,2, 0,3. Безриская ставка 10%, по базовому активу дивиденды не выплачиваются. Максимальная, минимальная цены базового актива и цена базового актива в текущий момент одинаковые и равны 100.

In [ ]:
# оценка справедливой стоимости лукбек опциона колл с фиксированным страйком
def FixedStrikeLookbackCall(S, K, S_max, T, q, r, sigma):

    if K <= S_max:
        e1 = (np.log(S/S_max) + (r-q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        e2 = e1 - sigma * np.sqrt(T)
        c_fix = np.exp(-r*T)*(S_max - K) + S * np.exp(-q*T)*norm.cdf(e1)-S_max*np.exp(-r*T)*norm.cdf(e2) + \
            S*np.exp(-r*T)*(sigma**2/(2*(r-q)))*(-np.power(S/S_max,-2*(r-q)/sigma**2) * norm.cdf(e1 - (2*(r-q)*np.sqrt(T)/sigma)) + np.exp((r-q)*T)*norm.cdf(e1))

        return c_fix

    elif K > S_max:
        d1 = (np.log(S/K) + (r-q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)

        c_fix = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2) + \
            S * np.exp(-r*T)*(sigma**2/(2*(r-q)))*(-np.power(S/K, -2*(r-q)/sigma**2) * norm.cdf(d1 - (2*(r-q)*np.sqrt(T)/sigma)) + np.exp((r-q)*T)*norm.cdf(d1))

        return c_fix

In [ ]:
# оценка справедливой стоимости лукбек опциона пут с фиксированным страйком
def FixedStrikeLookbackPut(S, K, Smin, T, q, r, sigma):

    if K < Smin:
        d1 = (np.log(S/K) + (r-q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        p_fix = K * np.exp(-r*T)*norm.cdf(-d2) - S * np.exp(-q*T)*norm.cdf(-d1) + \
            S * np.exp(-r*T)*(sigma**2/(2*(r-q))) * (np.power(S/K, -2*(r-q)/sigma**2)*norm.cdf(-d1 + 2 * (r-q)*np.sqrt(T)/sigma) - np.exp((r-q)*T)*norm.cdf(-d1))
        return p_fix

    elif K >= Smin:
        f1 = (np.log(S/S_min) + (r-q+sigma**2/2)*T)/(sigma*np.sqrt(T))
        f2 = f1 - sigma * np.sqrt(T)
        p_fix = (K - Smin)*np.exp(-r*T) - S * np.exp(-q*T)*norm.cdf(-f1) + Smin*np.exp(-r*T)*norm.cdf(-f2) +\
            S*np.exp(-r*T)*(sigma**2/(2*(r-q)))*(np.power(S/Smin, -2*(r-q)/sigma**2)*norm.cdf(-f1+2*(r-q)*np.sqrt(T)/sigma) - np.exp((r-q)*T) * norm.cdf(-f1))
        return p_fix

In [ ]:
price_function = {
    'call' : FixedStrikeLookbackCall,
    'put' : FixedStrikeLookbackPut
}

S = S_min = S_max = 100
q=0
r=0.1
prices_df = pd.DataFrame([
    {
        'T': T,
        'K': K,
        'sigma': sigma,
        'side': side,
        'price': price_function[side](S, K, S_min, T, q, r, sigma)
    }
    for T in [0.5, 1.0]
    for K in [95, 100, 105]
    for sigma in [0.1, 0.2, 0.3]
    for side in ['call', 'put']
])

In [ ]:
pd.pivot_table(prices_df, values='price', index=['T', 'K'], columns=['side', 'sigma'])

side          call                             put                      
sigma          0.1        0.2        0.3       0.1        0.2        0.3
T   K                                                                   
0.5 95   13.268722  18.926337  24.985760  0.689933   4.444777   8.921302
    100   8.512575  14.170190  20.229613  3.391665   8.317721  13.157880
    105   4.390794   9.890532  15.851199  8.147812  13.073868  17.914027
1.0 95   18.324153  26.073055  34.711618  1.053367   6.281345  12.237535
    100  13.799966  21.548868  30.187431  3.807895  10.129358  16.388857
    105   9.544490  17.296480  25.900190  8.332082  14.653545  20.913044